### Cartpole Problem

- There is a pole on a cart that can tip over if the cart moves too far left or right. 
- The problem is to find policies that do not tip the cart over and induce a loss. 
- The agent controls the cart and can take actions $a_t \in \{left,right\}$
- The environment is represented by a state $s_t$ that has 4 elements (position, velocity, etc.)
- The DQN is a neural network that predicts the action from a given state: $a_t = f(s_t;\theta)$.
- The environment is purely deterministic.
- Rewards are functions of the environment and time, $r_t = r(s_t)$ such that every incremental time step yeilds +1 and environment terminates if pole falls over or if cart moves too far away from center. 
- The Return at any state, is the discounted sum of all rewards (given policy and state transition): $R = \sum^{\infty}_{t=0} \gamma^t r_t$.
- The Q-function $Q(s,a)$ tells us the Return from being at state s and taking action a (and following optimal policy afterwards).
- The Goal is the find a policy $\pi$ which maps states to optimal actions i.e $a^* = \pi(s) = \text{argmax}_{a} Q(s,a)$. We will train the DQN to resemble this. 
- Every 

In [21]:
import gymnasium as gym
env = gym.make("CartPole-v1")

In [22]:
from collections import namedtuple, deque
Transition = namedtuple('Transition',('state','action','next_state','reward'))

In [23]:
import random

class ReplayMemory(object):
    # Holds transitions from experience and gives a random batch of transitions for training
    def __init__(self,capacity):
        self.memory = deque([], maxlen=capacity)
        
    def push(self,*args):
        "save transition"
        self.memory.append(Transition(*args))
        
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)
    
    

In [24]:
import torch.nn as nn
import torch.nn.functional as F

class DQN(nn.Module):
    # Input: difference between left and right screen patches
    # Output: Q(s,left) and Q(s,right)
    # Network is going to try to predict the expected Return of taking each action given the INPUT
    def __init__(self, n_obs, n_actions):
        super(DQN,self).__init__()
        self.layer1 = nn.Linear(n_obs, 128)
        self.layer2 = nn.Linear(128,128)
        self.layer3 = nn.Linear(128, n_actions)
    
    def forward(self,x):
        #print(x.shape)
        print(self.layer1(x).shape)
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)
        

In [25]:
BATCH_SIZE = 128 # number of transitions sampled from replay buffer
GAMMA = 0.99 # Discount factor
EPS_START = 0.9 # Initial value of Epsilon
EPS_END = 0.05 # End value of Epsilon
EPS_DECAY = 1000 # controls decay rate of Epsilon
TAU = 0.0005 # update rate of target network
LR = 1e-4 # Learning rate of Adam
n_actions = env.action_space.n # get number of actions from environment
state, info = env.reset() # load initial state from environment
n_obs = len(state) # get number of state observations
policy_net = DQN(n_obs,n_actions) # create Policy Net that will predict a given s
target_net = DQN(n_obs,n_actions) # 
target_net.load_state_dict(policy_net.state_dict())

import torch.optim as optim
optimizer = optim.AdamW(policy_net.parameters(),lr=LR,amsgrad=True)
memory=ReplayMemory(10000)

In [26]:
steps_done = 0

def select_action(state):
    # Given state selects action either from the DQN or randomly. 
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START-EPS_END)*math.exp(-1.*steps_done/EPS_DECAY)
    steps_done += 1
    
    if sample>eps_threshold:
        with torch.no_grad():
            return policy_net(state).max(1)[1].view(1,1) # pick up action with largest expected reward
            #return torch.max(policy_net(state),0)[1].view(1,1) # pick up action with largest expected reward
    else:
        return torch.tensor([[env.action_space.sample()]],dtype=torch.long)
episode_durations = []

import matplotlib
import matplotlib.pyplot as plt
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display


def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations,dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    
    if len(durations_t)>=100:
        means = durations_t.unfold(0,100,1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99),means))
        plt.plot(means.numpy())
        
    
    plt.pause(0.001) # pause to update plots
    
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())
            
    

In [29]:
def optimize_model():
    if len(memory)<BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))
    
    # final state is after simulation is done
    # these are the "s(t+1)" from the transitions
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)),dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
    
    # we begin with a batch of transitions - (state, action, reward)
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    
    # For these batch states we compute optimal policy according to our policy-network
    # Compute Q(s,a) and get actions - these are the actions that would have been taken for each batch state 
    print(state_batch.shape)
    state_action_values = policy_net(state_batch).gather(1,action_batch)
    
    # Compute V(s_t+1)
    next_state_values = torch.zeros(BATCH_SIZE)
    with torch.no_grad():
        # Use the "old" target_net to obtain value (Expected Return) on optimal actions from sampled s(t+1)
        # This is Q_old(s',a')
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0]
        
    # Here we have add r + max Q_old(s',a')
    expected_state_action_values = (next_state_values*GAMMA)+reward_batch

    # Compute the loss
    # Q_new(s,a) - r + max Q_old(s',a'): is the error
    # Huber loss function
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values,expected_state_action_values.unsqueeze(1))
    #print(loss.item())
        
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_value_(policy_net.parameters(),100)
    optimizer.step()
        
    

In [30]:
from itertools import count
import torch
import math

num_episodes = 200
for i_episode in range(num_episodes):
    
    # Draw from environment
    state, info = env.reset()
    state = torch.tensor(state,dtype=torch.float32).unsqueeze(0)
    
    # begin loop
    for t in count():
        action = select_action(state)
        obs, reward, terminated, truncated, _ = env.step(action.item())
        reward = torch.tensor([reward])
        done = terminated or truncated
        
        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(obs,dtype=torch.float32).unsqueeze(0)
        
        # store memory in transition
        memory.push(state,action,next_state,reward)
        
        # move to next state
        state = next_state
        
        # optimize
        optimize_model()
        
        # soft update target_net
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU+target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)
        
        if done:
            episode_durations.append(t+1)
            #plot_durations()
            break
            
    print('Complete')
    #plot_durations(show_result=True)
    plt.ioff()
    plt.show()


torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([115, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([120, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([117, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([118, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([118, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([111, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([111, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([114, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([121, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([119, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([111, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([118, 128])
Complete
torch.Size([1, 128]

torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([118, 128])
Complete
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([116, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([116, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([119, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([116, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([115, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([115, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([120, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([116, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([118, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([114, 128])
Complete
torch.Size([1, 128])
torch.Size([12

torch.Size([120, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([109, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([120, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([115, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([119, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([121, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([113, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([120, 128])
Complete
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([111, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([119, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([114, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([117, 128])
torch.Size([128, 4])


torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([120, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([117, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([115, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([118, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([114, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([116, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([120, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([119, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([119, 128])
Complete
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([118, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([117, 128])
torch.Size([1, 128])
torch.Size([128, 4])
to

torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([116, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([114, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([124, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([118, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([118, 128])
Complete
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([118, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([114, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([121, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([117, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([121, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([120, 128])
torch.Size([1, 128])
to

...

torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([125, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([128, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([127, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([125, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([125, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([124, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([127, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([127, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([126, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([127, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([128, 128])
torch.Size([1, 128])
torch.Size(

torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([126, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([125, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([126, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([125, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([126, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([123, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([125, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([126, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([127, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([127, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([128, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size(

torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([126, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([128, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([127, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([125, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([125, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([127, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([128, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([125, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([127, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([126, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([127, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size(

torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([126, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([124, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([124, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([124, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([127, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([126, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([126, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([124, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([126, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([124, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([125, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Siz

torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([127, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([126, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([127, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([126, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([128, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([126, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([126, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([127, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([126, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([125, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size([128, 128])
torch.Size([126, 128])
torch.Size([1, 128])
torch.Size([128, 4])
torch.Size(

In [ ]:
policy_net(state)

In [ ]:
%pip install gymnasium[classic_control]

In [ ]:
state, info = env.reset()
state = torch.tensor(state,dtype=torch.float32).unsqueeze(0)
print(state)
print(policy_net(state))